In [40]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
from time import sleep

from scrape import get_first_basket, getId

In [37]:
yst = datetime.today() - timedelta(days = 1)

url = f'https://www.basketball-reference.com/boxscores/?month={yst.month}&day={yst.day}&year={yst.year}'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

In [52]:
def predict_first_basket() :
    ######### starting_lineups #########
    return np.random.randint(0, 10)

In [61]:
game_ids = [getId(x) for x in soup.find_all('a', href = True) if 'boxscores/pbp' in x['href']]
dfs = []
for gameId in game_ids :
    sleep(3)
    df, starting_lineups = get_first_basket(gameId, starting_lineups = True)
    idx = predict_first_basket()
    df['first_basket_pred'] = (starting_lineups[0] + starting_lineups[1])[idx]
    dfs.append(df[['game_id', 'Home', 'Away', 'first_basket', 'first_basket_tm', 'first_basket_pred']])

first_basket_df = pd.concat(dfs).set_index('game_id')
first_basket_df['Date'] = yst.date()
first_basket_df['correct_pred'] = (first_basket_df['first_basket'] == first_basket_df['first_basket_pred'])
first_basket_df[['Date', 'Home', 'Away', 'first_basket', 'first_basket_tm', 'first_basket_pred']]

,game_id,Date,Home,Away,first_basket,first_basket_tm,first_basket_pred
0,202411100DEN,2024-11-10,DEN,DAL,murraja01,DEN,watsope01
1,202411100DET,2024-11-10,DET,HOU,stewais01,DET,smithja05
2,202411100IND,2024-11-10,IND,NYK,hartjo01,NYK,townska01
3,202411100LAL,2024-11-10,LAL,TOR,dickgr01,TOR,quickim01
4,202411100MIL,2024-11-10,MIL,BOS,antetgi01,MIL,brownja02
5,202411100MIN,2024-11-10,MIN,MIA,edwaran01,MIN,mcdanja02
6,202411100OKC,2024-11-10,OKC,GSW,dortlu01,OKC,willija06
7,202411100ORL,2024-11-10,ORL,WAS,coulibi01,WAS,bitadgo01
8,202411100PHI,2024-11-10,PHI,CHO,ballla01,CHO,greenjo02
9,202411100PHO,2024-11-10,PHO,SAC,dunnry01,PHO,jonesty01


In [64]:
first_basket_df = first_basket_df[['game_id', 'Date', 'Home', 'Away', 'first_basket', 'first_basket_tm', 'first_basket_pred', 'correct_pred']]

In [69]:
acc = first_basket_df['correct_pred'].mean()
print(f'Accuracy: {round(100 * acc, 1)}%  [{first_basket_df["correct_pred"].sum()}/{first_basket_df.shape[0]}]')
first_basket_df

Accuracy: 0.0%  [0/11]


,game_id,Date,Home,Away,first_basket,first_basket_tm,first_basket_pred,correct_pred
0,202411100DEN,2024-11-10,DEN,DAL,murraja01,DEN,watsope01,False
1,202411100DET,2024-11-10,DET,HOU,stewais01,DET,smithja05,False
2,202411100IND,2024-11-10,IND,NYK,hartjo01,NYK,townska01,False
3,202411100LAL,2024-11-10,LAL,TOR,dickgr01,TOR,quickim01,False
4,202411100MIL,2024-11-10,MIL,BOS,antetgi01,MIL,brownja02,False
5,202411100MIN,2024-11-10,MIN,MIA,edwaran01,MIN,mcdanja02,False
6,202411100OKC,2024-11-10,OKC,GSW,dortlu01,OKC,willija06,False
7,202411100ORL,2024-11-10,ORL,WAS,coulibi01,WAS,bitadgo01,False
8,202411100PHI,2024-11-10,PHI,CHO,ballla01,CHO,greenjo02,False
9,202411100PHO,2024-11-10,PHO,SAC,dunnry01,PHO,jonesty01,False
